In [1]:
# install packages - did this in command line conda environment
# remotes::install_github("jokergoo/circlize@9b21578")
# remotes::install_github("jokergoo/ComplexHeatmap@7d95ca5")
# remotes::install_github("immunogenomics/presto@31dc97f")
# remotes::install_github("jinworks/CellChat@88c2e13")
# BiocManager::install("GenomeInfoDbData")

In [1]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(zellkonverter)
    # library(scater)
    #library(scran)
    # library(scuttle)
    library(SingleCellExperiment)
    library(CellChat)
})

In [2]:
dist_out_dir <- "/home/workspace/spatial_mouse_lung_outputs/downstream_analysis/distance"

cellchat_out_dir <- file.path(dist_out_dir, "cellchat_distance")
if (!dir.exists(cellchat_out_dir)) {
  dir.create(cellchat_out_dir, recursive = TRUE)
}



In [3]:
sce = readH5AD(file.path(dist_out_dir, "adata_cellchat_distance_prepped.h5ad"))

Warning message:
“The names of these selected uns items have been modified to match R
conventions: '_scvi_manager_uuid' -> 'X_scvi_manager_uuid' and '_scvi_uuid' ->
'X_scvi_uuid'”
Warning message:
“The names of these selected obs columns have been modified to match R
conventions: '_scvi_batch' -> 'X_scvi_batch' and '_scvi_labels' ->
'X_scvi_labels'”


In [4]:
#sce$Timepoint <- stringr::str_extract(sce$batch, "\\d+")
sce$sample_label <- factor(sce$sample_label, levels = c("HDM_day3", "HDM_day30"))

In [5]:
reducedDimNames(sce)

[1] "X_pca"                  "X_scANVI"               "X_scVI_refalign"       
[4] "X_umap"                 "X_umap_scanvi_refalign" "X_umap_scvi_refalign"  
[7] "spatial"

In [6]:
run_cellchat <- function(sce_tmp, name, out_dir) {
    
    print("step: data.input"); flush.console()
    data.input = assay(sce_tmp, "X") # X are the log norm counts here, see part 1
    meta = data.frame(labels = sce_tmp$label_fine,
                    samples = sce_tmp$sample_label_cp, # (KA - I only have one sample, this is just a copy of sample_label
                    row.names = colnames(sce_tmp))
    print("step: spatial.locs"); flush.console()
    # spatial.locs = reducedDim(sce_tmp, 'spatial') |> as.data.frame() # KA this gives me an error 
    # spatial.locs = as.matrix(reducedDim(sce_tmp, 'spatial')) # this takes a long time
    spatial.locs = reducedDim(sce_tmp, 'spatial')  # try this
    scale.factors = list(spot.diameter = 5, spot = 5)
    spatial.factors = data.frame(ratio = 1, tol = 5)

    print("running: createCellChat")
    cellchat <-
        createCellChat(
            object = data.input,
            meta = meta,
            group.by = "labels",
            datatype = "spatial",
            coordinates = spatial.locs,
            spatial.factors = spatial.factors
        )


    CellChatDB <- CellChatDB.mouse # use CellChatDB.human if running on human data


    # use a subset of CellChatDB for cell-cell communication analysis
    # CellChatDB.use <- subsetDB(CellChatDB, search = "Secreted Signaling") # use Secreted Signaling
    # use all CellChatDB for cell-cell communication analysis
    CellChatDB.use <- CellChatDB # simply use the default CellChatDB

    # set the used database in the object
    cellchat@DB <- CellChatDB.use

    # subset the expression data of signaling genes for saving computation cost
    cellchat <- subsetData(cellchat) # This step is necessary even if using the whole database

    # future::plan("multisession", workers = 8) # do parallel
    print("running: identifyOverExpressedGenes, identifyOverExpressedInteractions")
    cellchat <- identifyOverExpressedGenes(cellchat)
    cellchat <- identifyOverExpressedInteractions(cellchat)

    # Typically, contact.range = 10, which is a typical human cell size
    print("running: computeCommunProb")
    cellchat <- computeCommunProb(cellchat,
        # type = "truncatedMean", trim = 0.1, # try 0.1, could lower to 0.05
        type = "truncatedMean", trim = 0.05, # try 0.1, could lower to 0.05
        distance.use = TRUE, interaction.range = 100,
        scale.distance = 1,
        contact.dependent = TRUE, contact.range = 10
    )
    # Filter out the cell-cell communication if there are only few number of cells in certain cell groups
    print("running: filterCommunication")
    cellchat <- filterCommunication(cellchat, min.cells = 10)

    print("running: computeCommunProbPathway")
    cellchat <- computeCommunProbPathway(cellchat)
    cellchat <- aggregateNet(cellchat)

    print(paste("running: saveRDS for sample: ", name))
    # saveRDS(cellchat, file = file.path(out_dir, paste0("cellchat_trim01_",name,".rds")))
    saveRDS(cellchat, file = file.path(out_dir, paste0("cellchat_trim05_",name,".rds")))
}

In [11]:
cellchat_sample <- function(sce, sample_label_select, out_dir) {

    sce_tmp = sce[,sce$sample_label == sample_label_select]

    # try on subset
    sce_tmp <- sce_tmp[, sample(1:ncol(sce_tmp), 1000)]
    
    # This is absolutely key. Otherwise Cellchat does not WORK!
    # KA note here - we have only one sample per condition - make a copy of the sample_label column 
    sce_tmp$sample_label_cp <- sce_tmp$sample_label
    sce_tmp$sample_label_cp <- droplevels(sce_tmp$sample_label_cp)
    # KA also do this for label column
    sce_tmp$label_fine <- droplevels(sce_tmp$label_fine)

    print("run_cellchat"); flush.console()
    run_cellchat(sce_tmp, sample_label_select, out_dir)
}

In [12]:
cellchat_out_dir

[1] "/home/workspace/spatial_mouse_lung_outputs/downstream_analysis/distance/cellchat_distance"

In [14]:
cellchat_sample(sce, 'HDM_day3', cellchat_out_dir)
cellchat_sample(sce, 'HDM_day30', cellchat_out_dir)


In [3]:
CellChatDB <- CellChatDB.mouse # use CellChatDB.human if running on human data

CellChatDB.use <- CellChatDB

In [6]:
head(CellChatDB.use$interaction)

,interaction_name,pathway_name,ligand,receptor,agonist,antagonist,co_A_receptor,co_I_receptor,evidence,annotation,⋯,receptor.symbol,receptor.family,receptor.location,receptor.keyword,receptor.surfaceome_main,receptor.surfaceome_sub,receptor.adhesome,receptor.secreted_type,receptor.transmembrane,version
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>
TGFB1_TGFBR1_TGFBR2,TGFB1_TGFBR1_TGFBR2,TGFb,Tgfb1,TGFbR1_R2,TGFb agonist,TGFb antagonist,,TGFb inhibition receptor,KEGG: mmu04350,Secreted Signaling,⋯,"Tgfbr1, Tgfbr2","Protein kinase superfamily, TKL Ser/Thr protein kinase","Cell membrane, Secreted, Membrane raft, Cell surface, Cell junction, Tight junction","Membrane, Secreted, Disulfide bond, Kinase, Transmembrane, Differentiation, ATP-binding, Receptor, Alternative splicing, Aortic aneurysm, Transferase, Glycoprotein, Manganese, Disease variant, Growth regulation, Signal, Reference proteome, Metal-binding, Transmembrane helix, Hereditary nonpolyposis colorectal cancer, Cell membrane, Phosphoprotein, Direct protein sequencing, Apoptosis, Craniosynostosis, Serine/threonine-protein kinase, 3D-structure, Nucleotide-binding, Magnesium, Tight junction, Isopeptide bond, Cell junction, Ubl conjugation",Receptors,Act.TGFB;Kinase,,,TRUE,CellChatDB v1
TGFB2_TGFBR1_TGFBR2,TGFB2_TGFBR1_TGFBR2,TGFb,Tgfb2,TGFbR1_R2,TGFb agonist,TGFb antagonist,,TGFb inhibition receptor,KEGG: mmu04350,Secreted Signaling,⋯,"Tgfbr1, Tgfbr2","Protein kinase superfamily, TKL Ser/Thr protein kinase","Cell membrane, Secreted, Membrane raft, Cell surface, Cell junction, Tight junction","Membrane, Secreted, Disulfide bond, Kinase, Transmembrane, Differentiation, ATP-binding, Receptor, Alternative splicing, Aortic aneurysm, Transferase, Glycoprotein, Manganese, Disease variant, Growth regulation, Signal, Reference proteome, Metal-binding, Transmembrane helix, Hereditary nonpolyposis colorectal cancer, Cell membrane, Phosphoprotein, Direct protein sequencing, Apoptosis, Craniosynostosis, Serine/threonine-protein kinase, 3D-structure, Nucleotide-binding, Magnesium, Tight junction, Isopeptide bond, Cell junction, Ubl conjugation",Receptors,Act.TGFB;Kinase,,,TRUE,CellChatDB v1
TGFB3_TGFBR1_TGFBR2,TGFB3_TGFBR1_TGFBR2,TGFb,Tgfb3,TGFbR1_R2,TGFb agonist,TGFb antagonist,,TGFb inhibition receptor,KEGG: mmu04350,Secreted Signaling,⋯,"Tgfbr1, Tgfbr2","Protein kinase superfamily, TKL Ser/Thr protein kinase","Cell membrane, Secreted, Membrane raft, Cell surface, Cell junction, Tight junction","Membrane, Secreted, Disulfide bond, Kinase, Transmembrane, Differentiation, ATP-binding, Receptor, Alternative splicing, Aortic aneurysm, Transferase, Glycoprotein, Manganese, Disease variant, Growth regulation, Signal, Reference proteome, Metal-binding, Transmembrane helix, Hereditary nonpolyposis colorectal cancer, Cell membrane, Phosphoprotein, Direct protein sequencing, Apoptosis, Craniosynostosis, Serine/threonine-protein kinase, 3D-structure, Nucleotide-binding, Magnesium, Tight junction, Isopeptide bond, Cell junction, Ubl conjugation",Receptors,Act.TGFB;Kinase,,,TRUE,CellChatDB v1
TGFB1_ACVR1B_TGFBR2,TGFB1_ACVR1B_TGFBR2,TGFb,Tgfb1,ACVR1B_TGFbR2,TGFb agonist,TGFb antagonist,,TGFb inhibition receptor,PMID: 27449815,Secreted Signaling,⋯,"Acvr1b, Tgfbr2","Protein kinase superfamily, TKL Ser/Thr protein kinase","Cell membrane, Secreted, Membrane raft","Membrane, Secreted, Disulfide bond, Kinase, Transmembrane, Differentiation, ATP-binding, Receptor, Alternative splicing, Aortic aneurysm, Transferase, Glycoprotein, Manganese, Disease variant, Growth regulation, Signal, Reference proteome, Metal-binding, Transmembrane helix, Hereditary nonpolyposis colorectal cancer, Cell membrane, Phosphoprotein, Direct protein sequencing, Apoptosis, Craniosynostosis, Serine/threonine-protein kinase, 3D-structure, Nucleotide-binding, Magnesium, Ubl conjugation",Receptors,Act.TGFB;Kinase,,,TRUE,CellChatDB v1
TGFB1_ACVR1C_TGFBR2,TGFB1_

In [18]:
filt <- CellChatDB.use$interaction %>%
  # filter(ligand == "Cd40" | receptor == "Cd40")
# filter(ligand == "Tgfb1" | receptor == "Tgfb1")
# filter(ligand == "Pdcd1lg1")
# filter(ligand == "Flt3l")
filter(ligand == "Cd274")
# filter(pathway_name == "PDL2")
filt

,interaction_name,pathway_name,ligand,receptor,agonist,antagonist,co_A_receptor,co_I_receptor,evidence,annotation,⋯,receptor.symbol,receptor.family,receptor.location,receptor.keyword,receptor.surfaceome_main,receptor.surfaceome_sub,receptor.adhesome,receptor.secreted_type,receptor.transmembrane,version
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>
PDCD1LG2_PDCD1,PDCD1LG2_PDCD1,PDL2,Pdcd1lg2,Pdcd1,,,,,PMID: 23954143,Cell-Cell Contact,⋯,Pdcd1,,Cell membrane,"Systemic lupus erythematosus, Immunity, Membrane, Disulfide bond, Transmembrane, Adaptive immunity, Glycoprotein, Signal, Reference proteome, Immunoglobulin domain, Cell membrane, Transmembrane helix, Phosphoprotein, Apoptosis, Isopeptide bond, 3D-structure, Ubl conjugation",Unclassified,Unclassified,,,TRUE,CellChatDB v1


In [43]:
CellChatDB.mouse$interaction %>%
  filter(ligand == "Il33" | receptor == "Il33")

,interaction_name,pathway_name,ligand,receptor,agonist,antagonist,co_A_receptor,co_I_receptor,evidence,annotation,⋯,receptor.symbol,receptor.family,receptor.location,receptor.keyword,receptor.surfaceome_main,receptor.surfaceome_sub,receptor.adhesome,receptor.secreted_type,receptor.transmembrane,version
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>
IL33_IL1RL1_IL1RAP,IL33_IL1RL1_IL1RAP,IL1,Il33,IL1RL1_IL1RAP,,,,,KEGG: mmu04060,Secreted Signaling,⋯,"Il1rl1, Il1rap",Interleukin-1 receptor,"Cell membrane, Secreted","Immunity, Membrane, Secreted, Disulfide bond, Transmembrane, Receptor, Alternative splicing, Glycoprotein, Hydrolase, Reference proteome, Signal, Immunoglobulin domain, Cell membrane, Innate immunity, Transmembrane helix, Phosphoprotein, 3D-structure, NAD, Inflammatory response, Repeat, Direct protein sequencing",Receptors,CytokineR;IG;Type1,,cytokine,TRUE,CellChatDB v1
